In [1]:
import os
os.environ['GEMINI_API_KEY'] = 'xxxxxxxx'

In [2]:
import pandas as pd

TASKNAMES = ["subtask1", "subtask2"]
BATCHNAMES = ["batch1", "batch2", "batch3"]

dataset_loader = dict()

for taskname in TASKNAMES:
    for batchname in BATCHNAMES:
        print(f"Processing {taskname} for {batchname}")
        prefix = "dataset/"
        # Load the data
        data = pd.read_csv(f"{prefix}/{taskname}_train_{batchname}.csv")
        dataset_loader[f"{taskname}_{batchname}"] = data
        

d:\software\minianaconda\envs\ag_llama\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Processing subtask1 for batch1
Processing subtask1 for batch2
Processing subtask1 for batch3
Processing subtask2 for batch1
Processing subtask2 for batch2
Processing subtask2 for batch3


### Statistics

Hereby I list the number of each unique answers and claims for each batch datatset

| Task | Batch | number of answers | number of claims |
| ---  | ----- | ----------------- | ---------------- |
| 1    | 1     | 235               | 500              |
| 1    | 2     | 468               | 2092             |
| 1    | 3     | 469               | 1500             |


(seems that subtask 1 and 2 have the shared dataset)
| Task | Batch | number of answers | number of claims |
| ---  | ----- | ----------------- | ---------------- |
| 2    | 1     | 235               | 500              |
| 2    | 2     | 468               | 2092             |
| 2    | 3     | 469               | 1500             |

In [3]:
all_answer = dataset_loader["subtask2_batch2"].answer.unique()

import os

os.makedirs("outputs/statistics&collections", exist_ok=True)

with open("outputs/statistics&collections/all_answer.txt", "w") as f:
    for answer in all_answer:
        f.write(answer + "\n" + "-"*100 + "\n")


# Testing to get named entity in reference

Run `process_text_to_knowledge_graph` function on reference with *gemini-2.5-flash* model.

Now we use gemini-2.5 flash model and you can use it for free. Register at google vertext AI and create an API key, put it in `.env` file in the same folder with `GEMINI_API_KEY=...`.

In [4]:
import asyncio
import os
import networkx as nx

from dotenv import load_dotenv
from operationCheatSheet import chunking_by_token_size, extract_entities
from utils import compute_mdhash_id
from geminillm import gemini_complete_if_cache
from openaillm import openai_embed, openai_complete
from faiss_impl import FaissVectorDBStorage
from networkx_impl import NetworkXStorage
from json_kv_iml import JsonKVStorage
from shared_storage import initialize_share_data
from cheatsheet import CHEATSHEETS


# LLM_MODEL_NAME = "gpt-3.5-turbo"
LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"

load_dotenv()

async def process_text_to_knowledge_graph(text_content,
                                          cheatsheet_knowledge_graph_inst,
                                          file_path="unknown_source"):
    # Step 1: Create chunks from your text content
    raw_chunks = chunking_by_token_size(
        content=text_content,
        split_by_character=None,  # Optional: specify a character to split by (e.g., "\n\n")
        split_by_character_only=False,
        overlap_token_size=128,
        max_token_size=1024,
        tiktoken_model=LLM_MODEL_NAME
    )
    
    # Step 2: Format chunks as required by extract_entities
    doc_id = compute_mdhash_id(text_content, prefix="doc-")
    chunks = {
        compute_mdhash_id(chunk["content"], prefix="chunk-"): {
            **chunk,
            "full_doc_id": doc_id,
            "file_path": file_path
        }
        for chunk in raw_chunks
    }
    
    # Step 3: Initialize your storage instances
    faiss_global_config = {
        "working_dir": "/tmp",
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    kv_global_config = {
        "working_dir": "/tmp",
        "llm_model_name": LLM_MODEL_NAME,
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    knowledge_graph = NetworkXStorage(namespace="nx_kg", global_config=faiss_global_config, embedding_func=openai_embed)  # Graph storage implementation
    entity_vector_db = FaissVectorDBStorage(namespace="faiss_entity", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for entities
    relationship_vector_db = FaissVectorDBStorage(namespace="faiss_relationship", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for relationships
    llm_cache = JsonKVStorage(namespace="llm_cache", global_config=kv_global_config, embedding_func=openai_embed)
    
    initialize_share_data()

    await knowledge_graph.initialize()
    await entity_vector_db.initialize()
    await relationship_vector_db.initialize()
    await llm_cache.initialize()  # if your cache supports/needs it

    # Step 4: Use the OpenAI LLM function from openai.py
    # Create a wrapper that matches the expected signature for extract_entities
    async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
        if history_messages is None:
            history_messages = []
    

        # Use Google GenAI
        return await gemini_complete_if_cache(
            model=LLM_MODEL_NAME,
            prompt=prompt,
            history_messages=history_messages,
            hashing_kv=llm_cache,
            temperature=0.2,
            max_tokens=max_tokens or 1024,
        )

        
    
    # Step 5: Configure the extraction process
    global_config = {
        "llm_model_func": llm_wrapper,  # Use our wrapper function
        "llm_model_name": LLM_MODEL_NAME,  # Required for some OpenAI functions
        "entity_extract_max_gleaning": 2,
        "force_llm_summary_on_merge": False,
        "llm_model_max_token_size": 1024,
        "summary_to_max_tokens": 256,
        "tiktoken_model_name": LLM_MODEL_NAME,
        "addon_params": {
            "language": "English",
            "entity_types": [
                "Software or Computational Method",
                "Scientific Method",
                "Infrastructure or Equipment",
                "Institution or Organization",
                "Ecological or Biological Entity",
                "Pollutant or Material",
                "Time Expression",
                "Health or Disease Concept",
                "Food or Nutrition Element",
                "Demographic Group",
                "Measurement or Quantity"
            ]
        },
    }
    
    # Step 6: Optional status tracking
    pipeline_status = {
        "latest_message": "",
        "history_messages": []
    }
    pipeline_status_lock = asyncio.Lock()
    
    # Step 7: Run the extraction
    await extract_entities(
        chunks=chunks,
        knowledge_graph_inst=knowledge_graph,
        entity_vdb=entity_vector_db,
        relationships_vdb=relationship_vector_db,
        global_config=global_config,
        pipeline_status=pipeline_status,
        pipeline_status_lock=pipeline_status_lock,
        llm_response_cache=llm_cache,
        cheatsheet_knowledge_graph_inst=cheatsheet_knowledge_graph_inst,
        write_result_to_txt=True,
        special_interest=CHEATSHEETS["special_interests"]
    )
    
    return knowledge_graph


# Example usage in a Jupyter notebook
async def main(content, nx_graph):
    # monkeyReader = reader.MonkeyReader('monkey') # or 'lxml' or 'x2d'

    # read paper content
    # essay = monkeyReader.readEssay('./xmldata/Byjesh.pdf.tei.xml')
    # text = "\n".join(essay["Impact of fixed rise in temperature, CO 2 and change in rain fall"])
    text = content
    # nx_graph = nx.Graph()
    knowledge_graph = await process_text_to_knowledge_graph(text, nx_graph, "example.txt")
    # Now you can query the knowledge graph for entities and relationships
    
    return knowledge_graph

d:\software\minianaconda\envs\ag_llama\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-16 00:59:46 - pipmaster.package_manager - INFO - Targeting pip associated with Python: d:\software\minianaconda\envs\ag_llama\python.exe | Command base: d:\software\minianaconda\envs\ag_llama\python.exe -m pip


In [5]:
def create_knowledge_graph_from_results(nodes, edges) -> nx.Graph:
    """
    Creates a NetworkX graph from the extracted nodes and edges.
    Returns:
        nx.Graph: Graph containing entities as nodes and their relationships
    """
    G = nx.Graph()
    
    # Add nodes from the extracted results
    for entity_name, entity_list in nodes.items():
        for entity in entity_list:
            G.add_node(
                entity["entity_name"],
                entity_type=entity.get("entity_type", ""),
                description=entity.get("description", ""),
                source_id=entity.get("source_id", ""),
                file_path=entity.get("file_path", "")
            )
    
    # Add edges from the extracted results
    for edge_key, edge_list in edges.items():
        for edge in edge_list:
            source = edge.get("src_id", "")
            target = edge.get("tgt_id", "")
            if source and target:
                G.add_edge(
                    source,
                    target,
                    description=edge.get("description", ""),
                    keywords=edge.get("keywords", ""),
                    weight=edge.get("weight", 1.0),
                    source_id=edge.get("source_id", ""),
                    file_path=edge.get("file_path", "")
                )
    
    return G

def compare_knowledge_graphs(claim_graph, reference_graph):
    """
    Compare claim graph with reference graph to determine entailment relationship.
    
    Args:
        claim_graph: NetworkX graph from claim
        reference_graph: NetworkX graph from reference
    
    Returns:
        str: "Entailment", "Contradiction", or "Unverifiable"
    """
    # Get nodes from both graphs
    claim_nodes = set(claim_graph.nodes())
    reference_nodes = set(reference_graph.nodes())
    
    # Check if all claim nodes exist in reference graph
    nodes_match = claim_nodes.issubset(reference_nodes)
    
    if not nodes_match:
        # If nodes don't match, it's unverifiable
        return "unver"
    
    # If nodes match, check edges with their connected nodes
    claim_edges_with_nodes = set()
    reference_edges_with_nodes = set()
    
    # Extract edges with their node information for claim graph
    for edge in claim_graph.edges():
        source, target = edge
        # Create a tuple that represents the edge with its nodes
        claim_edges_with_nodes.add((source, target))
        # Also add the reverse direction to handle undirected graphs
        claim_edges_with_nodes.add((target, source))
    
    # Extract edges with their node information for reference graph
    for edge in reference_graph.edges():
        source, target = edge
        # Create a tuple that represents the edge with its nodes
        reference_edges_with_nodes.add((source, target))
        # Also add the reverse direction to handle undirected graphs
        reference_edges_with_nodes.add((target, source))
    
    # Check if all claim edges (with their nodes) exist in reference graph
    edges_match = all(edge in reference_edges_with_nodes for edge in claim_edges_with_nodes)
    
    if edges_match:
        # Both nodes and edges match - entailment
        return "entail"
    else:
        # Nodes match but edges don't - contradiction
        return "contra"


132

In [7]:
from utils import get_latest_result, read_knowledge_graph_from_pickle
from sklearn.metrics import f1_score, classification_report
import pandas as pd

# Process all rows in batch2 and batch3
results = []

# Process batch2
print("Processing batch2...")
batch2_data = dataset_loader["subtask1_batch2"]
for idx, row in batch2_data.iterrows():
    try:
        reference_content = row.reference
        claim_content = row.claim
        row_id = row.ID
        true_label = row.label
        
        # Create knowledge graph from reference
        knowledge_graph_from_reference = await main(reference_content, nx.Graph())
        nodes_reference, edges_reference = get_latest_result()
        knowledge_graph_from_reference = create_knowledge_graph_from_results(nodes_reference, edges_reference)
        
        # Create knowledge graph from claim
        knowledge_graph_from_claim = await main(claim_content, knowledge_graph_from_reference)
        nodes_claim, edges_claim = get_latest_result()
        knowledge_graph_from_claim = create_knowledge_graph_from_results(nodes_claim, edges_claim)
        
        # Compare the two knowledge graphs
        predicted_label = compare_knowledge_graphs(knowledge_graph_from_claim, knowledge_graph_from_reference)
        
        results.append({
            'ID': row_id,
            'true_label': true_label,
            'predicted_label': predicted_label,
            'batch': 'batch2'
        })
        
        print(f"Processed {row_id}: True={true_label}, Predicted={predicted_label}")
        
    except Exception as e:
        print(f"Error processing row {row_id}: {e}")
        results.append({
            'ID': row_id,
            'true_label': true_label,
            'predicted_label': 'unver',  # Default to unverifiable on error
            'batch': 'batch2'
        })

# Process batch3 if available
if "subtask1_batch3" in dataset_loader:
    print("\nProcessing batch3...")
    batch3_data = dataset_loader["subtask1_batch3"]
    for idx, row in batch3_data.iterrows():
        try:
            reference_content = row.reference
            claim_content = row.claim
            row_id = row.ID
            true_label = row.label
            
            # Create knowledge graph from reference
            knowledge_graph_from_reference = await main(reference_content, nx.Graph())
            nodes_reference, edges_reference = get_latest_result()
            knowledge_graph_from_reference = create_knowledge_graph_from_results(nodes_reference, edges_reference)
            
            # Create knowledge graph from claim
            knowledge_graph_from_claim = await main(claim_content, knowledge_graph_from_reference)
            nodes_claim, edges_claim = get_latest_result()
            knowledge_graph_from_claim = create_knowledge_graph_from_results(nodes_claim, edges_claim)
            
            # Compare the two knowledge graphs
            predicted_label = compare_knowledge_graphs(knowledge_graph_from_claim, knowledge_graph_from_reference)
            
            results.append({
                'ID': row_id,
                'true_label': true_label,
                'predicted_label': predicted_label,
                'batch': 'batch3'
            })
            
            print(f"Processed {row_id}: True={true_label}, Predicted={predicted_label}")
            
        except Exception as e:
            print(f"Error processing row {row_id}: {e}")
            results.append({
                'ID': row_id,
                'true_label': true_label,
                'predicted_label': 'unver',  # Default to unverifiable on error
                'batch': 'batch3'
            })

# Convert results to DataFrame
results_df = pd.DataFrame(results)


No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 39704 Shared-Data created for Single Process
INFO: Process 39704 initialized updated flags for namespace: [nx_kg]
INFO: Process 39704 initialized updated flags for namespace: [faiss_entity]
INFO: Process 39704 initialized updated flags for namespace: [faiss_relationship]
INFO: Process 39704 initialized updated flags for namespace: [llm_cache]
INFO: Process 39704 ready to initialize storage namespace: [llm_cache]


Processing batch2...


e:\kgml-proj\Ag_Llama\agent_info_ext\LLM_meta_example\meta-knowledge-harvesting-llm\kaggle\operationCheatSheet.py:729: RuntimeWarning: coroutine 'extract_entities.<locals>.nightly_kg_to_text' was never awaited
  hint_prompt = fill_nightly_prompt.format(


Final result:
("entity"<|>"Chatbots"<|>"Software or Computational Method"<|>"The widespread use of chatbots is a reality and their application in higher education is promising."<|>"The widespread use of chatbots is a reality"<|>"[3]"<|> "")##
("entity"<|>"Higher Education"<|>"Institution or Organization"<|>"The widespread use of chatbots is a reality and their application in higher education is promising. Understanding higher education users’ expectations for the use of chatbots in education is important for the design and development of new solutions."<|>"their application in higher education is promising"<|>"[3]"<|> "")##
("entity"<|>"Pedagogical Uses of Chatbots"<|>"Software or Computational Method"<|>"The present investigation documents how higher education users envision the pedagogical uses of chatbots in higher education, and how experts in the domain of education chatbots perceive the potential benefits and challenges related to the use of chatbots in education."<|>"the pedagog

No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 39704 Shared-Data already initialized (multiprocess=False)
INFO: Process 39704 storage namespace already initialized: [llm_cache]


...

No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 39704 Shared-Data already initialized (multiprocess=False)
INFO: Process 39704 storage namespace already initialized: [llm_cache]


Final result:
("entity"<|>"Active Learning"<|>"Active Learning"<|>"Software or Computational Method"<|>"Active learning can also involve creating meta-examples that store features of learning problems and the performance of algorithms, which helps in predicting the best algorithms for new tasks [5]."|<|>"Active learning"|<|><|>)##
("entity"<|>"Meta-Examples"<|>"Meta-Examples"<|>"Software or Computational Method"<|>"Active learning can also involve creating meta-examples that store features of learning problems and the performance of algorithms, which helps in predicting the best algorithms for new tasks [5]."|<|>"meta-examples"|<|><|>)##
("entity"<|>"Algorithms"<|>"Algorithms"<|>"Software or Computational Method"<|>"Active learning can also involve creating meta-examples that store features of learning problems and the performance of algorithms, which helps in predicting the best algorithms for new tasks [5]."|<|>"algorithms"|<|><|>)##
("entity"<|>"Learning Problems"<|>"Learning Proble

No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 39704 Shared-Data already initialized (multiprocess=False)
INFO: Process 39704 storage namespace already initialized: [llm_cache]


Processed s_254: True=entail, Predicted=unver
Final result:
("entity"<|>"Blockchain"<|>"Blockchain"<|>"Software or Computational Method"<|>"Blockchain is an emerging technology that would possibly disrupt the existing centralized financial systems lead to the rise to a new technology era for the financial sector. Additionally, different new use cases such as healthcare, identity management, etc. suggest that Blockchain has much wider applications."<|>"Blockchain is an emerging technology"<|>"1"<|>"")##
("entity"<|>"Centralized Financial Systems"<|>"Centralized Financial Systems"<|>"Infrastructure or Equipment"<|>"Blockchain is an emerging technology that would possibly disrupt the existing centralized financial systems lead to the rise to a new technology era for the financial sector."<|>"existing centralized financial systems"<|>"1"<|>"")##
("entity"<|>"Financial Sector"<|>"Financial Sector"<|>"Institution or Organization"<|>"Blockchain is an emerging technology that would possibly di

CancelledError: 

In [12]:
# Save results_df to local file
import os
from datetime import datetime

# Create output directory if it doesn't exist
output_dir = "outputs/kaggle_results"
os.makedirs(output_dir, exist_ok=True)

# Generate timestamp for filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save as CSV
csv_filename = f"kaggle_evaluation_results_{timestamp}.csv"
csv_filepath = os.path.join(output_dir, csv_filename)
results_df.to_csv(csv_filepath, index=False)
print(f"Results saved to CSV: {csv_filepath}")

# Save as JSON for better readability
json_filename = f"kaggle_evaluation_results_{timestamp}.json"
json_filepath = os.path.join(output_dir, json_filename)
results_df.to_json(json_filepath, orient='records', indent=2)
print(f"Results saved to JSON: {json_filepath}")

# Save as pickle for preserving data types
pickle_filename = f"kaggle_evaluation_results_{timestamp}.pkl"
pickle_filepath = os.path.join(output_dir, pickle_filename)
results_df.to_pickle(pickle_filepath)
print(f"Results saved to Pickle: {pickle_filepath}")


Results saved to CSV: outputs/kaggle_results\kaggle_evaluation_results_20250616_080157.csv
Results saved to JSON: outputs/kaggle_results\kaggle_evaluation_results_20250616_080157.json
Results saved to Pickle: outputs/kaggle_results\kaggle_evaluation_results_20250616_080157.pkl


In [10]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate evaluation metrics
true_labels = results_df['true_label'].tolist()
predicted_labels = results_df['predicted_label'].tolist()

# Calculate weighted F1 score (main evaluation metric)
weighted_f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"\n=== EVALUATION RESULTS ===")
print(f"Total samples processed: {len(results_df)}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")

# Calculate accuracy
accuracy = (results_df['true_label'] == results_df['predicted_label']).mean()
print(f"Accuracy: {accuracy:.4f}")

# Print detailed classification report
print(f"\n=== DETAILED CLASSIFICATION REPORT ===")
print(classification_report(true_labels, predicted_labels))

# Show results by batch
print(f"\n=== RESULTS BY BATCH ===")
for batch in results_df['batch'].unique():
    batch_results = results_df[results_df['batch'] == batch]
    batch_accuracy = (batch_results['true_label'] == batch_results['predicted_label']).mean()
    batch_f1 = f1_score(batch_results['true_label'], batch_results['predicted_label'], average='weighted')
    print(f"{batch}: Accuracy={batch_accuracy:.4f}, Weighted F1={batch_f1:.4f}")

# Calculate F1 scores for batch2 and batch3 separately
print(f"\n=== INDIVIDUAL BATCH F1 SCORES ===")
if 'batch2' in results_df['batch'].values:
    batch2_results = results_df[results_df['batch'] == 'batch2']
    batch2_f1 = f1_score(batch2_results['true_label'], batch2_results['predicted_label'], average='weighted')
    print(f"Batch2 Weighted F1 Score: {batch2_f1:.4f}")

if 'batch3' in results_df['batch'].values:
    batch3_results = results_df[results_df['batch'] == 'batch3']
    batch3_f1 = f1_score(batch3_results['true_label'], batch3_results['predicted_label'], average='weighted')
    print(f"Batch3 Weighted F1 Score: {batch3_f1:.4f}")

# Display first few results
print(f"\n=== SAMPLE RESULTS ===")
print(results_df.head(10))



=== EVALUATION RESULTS ===
Total samples processed: 132
Weighted F1 Score: 0.1000
Accuracy: 0.2500

=== DETAILED CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

      contra       0.00      0.00      0.00        48
      entail       0.00      0.00      0.00        51
       unver       0.25      1.00      0.40        33

    accuracy                           0.25       132
   macro avg       0.08      0.33      0.13       132
weighted avg       0.06      0.25      0.10       132


=== RESULTS BY BATCH ===
batch2: Accuracy=0.2500, Weighted F1=0.1000

=== INDIVIDUAL BATCH F1 SCORES ===
Batch2 Weighted F1 Score: 0.1000

=== SAMPLE RESULTS ===
     ID true_label predicted_label   batch
0   s_4     contra           unver  batch2
1   s_5     contra           unver  batch2
2   s_7     contra           unver  batch2
3   s_8      unver           unver  batch2
4   s_9     contra           unver  batch2
5  s_10      unver           unver  batch2
6  s_13      un

d:\software\minianaconda\envs\ag_llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\software\minianaconda\envs\ag_llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\software\minianaconda\envs\ag_llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [ ]:
# Print detailed comparison information
print(f"\nClaim graph nodes: {list(knowledge_graph_from_claim.nodes())}")
print(f"Reference graph nodes: {list(knowledge_graph_from_reference.nodes())}")
print(f"Claim graph edges: {list(knowledge_graph_from_claim.edges())}")
print(f"Reference graph edges: {list(knowledge_graph_from_reference.edges())}")

# Print node matching details
claim_nodes = set(knowledge_graph_from_claim.nodes())
reference_nodes = set(knowledge_graph_from_reference.nodes())
missing_nodes = claim_nodes - reference_nodes
if missing_nodes:
    print(f"Missing nodes in reference: {missing_nodes}")
else:
    print("All claim nodes found in reference graph")

# Print edge matching details
claim_edges = set(knowledge_graph_from_claim.edges())
reference_edges = set(knowledge_graph_from_reference.edges())
missing_edges = claim_edges - reference_edges
if missing_edges:
    print(f"Missing edges in reference: {missing_edges}")
else:
    print("All claim edges found in reference graph")
